<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
ELEC PRICE PREDICTION CAPSTONE:</font>  

<font size='4'>NOAA CA Weather DataFrame Creation Notebook</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Manu Kalia Project Submission<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; DSI-7-SF<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15-May-2019<br>
</font>

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Declare-Variables,-Directories,-and-Column-Names" data-toc-modified-id="Declare-Variables,-Directories,-and-Column-Names-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Declare Variables, Directories, and Column Names</a></span></li><li><span><a href="#Define-Functions" data-toc-modified-id="Define-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Functions</a></span><ul class="toc-item"><li><span><a href="#Function-to-create-NOAA-weather-station-DataFrame" data-toc-modified-id="Function-to-create-NOAA-weather-station-DataFrame-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Function to create NOAA weather station DataFrame</a></span></li><li><span><a href="#Function-to-extract-relevant-statistics-from-NOAA-column" data-toc-modified-id="Function-to-extract-relevant-statistics-from-NOAA-column-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Function to extract relevant statistics from NOAA column</a></span></li><li><span><a href="#Function-to-print-relevant-descriptive-info-on-dataframe" data-toc-modified-id="Function-to-print-relevant-descriptive-info-on-dataframe-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Function to print relevant descriptive info on dataframe</a></span></li><li><span><a href="#Function-to-replace--hard-coded-&quot;99xxx9&quot;-values-w/-NaNs" data-toc-modified-id="Function-to-replace--hard-coded-&quot;99xxx9&quot;-values-w/-NaNs-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Function to replace  hard-coded "99xxx9" values w/ NaNs</a></span></li></ul></li><li><span><a href="#Read-in-Data-and-Create-4-Weather-Station-DataFrames" data-toc-modified-id="Read-in-Data-and-Create-4-Weather-Station-DataFrames-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read in Data and Create 4 Weather Station DataFrames</a></span></li><li><span><a href="#San-Diego-DataFrame" data-toc-modified-id="San-Diego-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>San Diego DataFrame</a></span></li><li><span><a href="#Riverside-DataFrame" data-toc-modified-id="Riverside-DataFrame-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Riverside DataFrame</a></span></li><li><span><a href="#Redding-DataFrame" data-toc-modified-id="Redding-DataFrame-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Redding DataFrame</a></span></li><li><span><a href="#Fresno-DataFrame" data-toc-modified-id="Fresno-DataFrame-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Fresno DataFrame</a></span></li></ul></div>

---

## Imports

In [1]:
import pandas as pd
import numpy as np

import wget, os
import time
import glob

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
sns.set_style('whitegrid')

---

## Declare Variables, Directories, and Column Names

In [2]:
stations_dict = {'72290023188': 'sand',        # San Diego
                 '72286903171': 'rive',        # Riverside
                 '72592024257': 'redd',        # Redding
                 '72389093193': 'fres'         # Fresno
                }

stn_id_list   = list(stations_dict.keys())
stn_name_list = list(stations_dict.values())

print(f'NOAA Weather Station IDs:   {stn_id_list}')
print(f'NOAA Weather Station Names: {stn_name_list}')

# Dataframe names
sand_df = None
rive_df = None
redd_df = None
fres_df = None

NOAA Weather Station IDs:   ['72290023188', '72286903171', '72592024257', '72389093193']
NOAA Weather Station Names: ['sand', 'rive', 'redd', 'fres']


In [3]:
download_dirs = ['../raw_data/noaa_weather/2016/',
                 '../raw_data/noaa_weather/2017/',
                 '../raw_data/noaa_weather/2018/',
                 '../raw_data/noaa_weather/2019/'
                ]

In [4]:
weather_orig_cols=['STATION',
                   'DATE',
                   'TMP',
                   'WND',
                   'CIG',
                   'VIS']

weather_new_cols =['stn_id',
                   'datetime',
                   'temp',
                   'wind',
                   'ceil',
                   'vis']

weather_rename_dict = {old: new for old, new in zip(weather_orig_cols,
                                                    weather_new_cols)}

---

##  Define Functions  

### Function to create NOAA weather station DataFrame

Parameters:  dataframe (df), index slice position in `stations_list`  

Function steps:  
 1. Instantiate DataFrame
 2. Loop through the 4 annual download directories
 3. Reading in data from .csv files, appending rows to new dataframe
 3. Sort by `datetime` column, cast to 'datetime' object, and set as index 
 4. Sort by new `datetime` index to show rows in chronological order
 
 
### Function to extract relevant statistics from NOAA column

Parameters:  dataframe (df), orig_col_name (string), new_col_name (string), slice_index  

Function steps:  
 1. Step through rows of a dataframe
 2. Extract a single statistic from a "cell" that is a tuple of strings
 3. Store that stat in a new column whose name is a parameter
 4. Drop the original data column from the dataframe
 
 
### Function to print relevant descriptive info on dataframe

Parameter:  dataframe (df)

Function steps:
 1. print df.shape
 2. print null counts
 3. print head(5)
 
 
 ### Function to replace  hard-coded "99xxx9" values w/ NaNs

Parameter:  dataframe (df)

Function steps:
 1. replace specific codes (depends on which column) NOAA uses as NaN
 2. return null counts

In [5]:
def create_stn_df(df, stn_idx):
    df       = pd.DataFrame(columns=weather_new_cols)
    stn_name = list(stations_dict.values())[stn_idx]
    stn_id   = list(stations_dict.keys())[stn_idx]

    for directory in download_dirs:
        file = directory + stn_id + '.csv'
        temp_df = pd.read_csv(file,
                              usecols=weather_orig_cols).rename(index=str,
                                                                columns=weather_rename_dict)
        df = df.append(temp_df, ignore_index=True, sort=True)

    df = df.sort_values(by='datetime').reset_index(drop=True)

    df['datetime'] = pd.to_datetime(df['datetime'])

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    print(f'{stn_name}_df DataFrame Created')
    
    return df

In [6]:
def extract_statistic(df, orig_col_name, new_col_name, slice_index):
    for row in range(0, df.shape[0]):
        try:
            statistic = df.loc[df.index[row], orig_col_name].split(',')[slice_index]
            df.loc[df.index[row], new_col_name] = statistic
            
        except IndexError: break
        except:
            df.drop(labels=df.index[row], axis=0, inplace=True)  

    df.drop(columns=[orig_col_name], inplace=True)

    return

In [7]:
def summarize(df):
    print(f'\nShape: {df.shape}')
    print(f'\n  Nulls:\n{df.isna().sum()}')
    print(f'\ndf.head()\n{df.head()}')
    
    return

In [8]:
def replace_nines(df):
    df.replace('9999', np.nan, inplace=True)
    df.replace('99999', np.nan, inplace=True)
    df.replace('999999', np.nan, inplace=True)
    df.isna().sum()
    
    return

---

## Read in Data and Create 4 Weather Station DataFrames

In [9]:
stn_name_list

['sand', 'rive', 'redd', 'fres']

In [10]:
sand_df = create_stn_df(sand_df, 0)
rive_df = create_stn_df(rive_df, 1)
redd_df = create_stn_df(redd_df, 2)
fres_df = create_stn_df(fres_df, 3)

sand_df DataFrame Created
rive_df DataFrame Created
redd_df DataFrame Created
fres_df DataFrame Created


---

## San Diego DataFrame

In [11]:
summarize(sand_df)


Shape: (41373, 5)

  Nulls:
ceil      0
stn_id    0
temp      0
vis       0
wind      0
dtype: int64

df.head()
                            ceil       stn_id     temp           vis  \
datetime                                                               
2016-01-01 00:00:00  99999,9,9,N  72290023188  +0161,1  016000,1,9,9   
2016-01-01 00:51:00  22000,5,9,N  72290023188  +0156,5  016093,5,N,5   
2016-01-01 01:51:00  22000,5,9,N  72290023188  +0144,5  016093,5,N,5   
2016-01-01 02:51:00  22000,5,9,N  72290023188  +0139,5  016093,5,N,5   
2016-01-01 03:51:00  22000,5,9,N  72290023188  +0133,5  016093,5,N,5   

                               wind  
datetime                             
2016-01-01 00:00:00  310,1,N,0031,1  
2016-01-01 00:51:00  320,5,N,0026,5  
2016-01-01 01:51:00  340,5,N,0021,5  
2016-01-01 02:51:00  999,9,C,0000,5  
2016-01-01 03:51:00  999,9,C,0000,5  


In [12]:
sand_df.drop_duplicates().shape

(26409, 5)

In [13]:
sand_df = sand_df.drop_duplicates()
sand_df.shape

(26409, 5)

In [14]:
extract_statistic(sand_df, 'temp', 'sand_temp', 0)
extract_statistic(sand_df, 'wind', 'sand_wind', 3)
extract_statistic(sand_df, 'vis',  'sand_vis',  0)
extract_statistic(sand_df, 'ceil', 'sand_ceil', 0)

sand_df.drop(columns=['stn_id'], inplace=True)

In [15]:
summarize(sand_df)


Shape: (26397, 4)

  Nulls:
sand_temp    6
sand_wind    0
sand_vis     0
sand_ceil    0
dtype: int64

df.head()
                    sand_temp sand_wind sand_vis sand_ceil
datetime                                                  
2016-01-01 00:00:00     +0161      0031   016000     99999
2016-01-01 00:51:00     +0156      0026   016093     22000
2016-01-01 01:51:00     +0144      0021   016093     22000
2016-01-01 02:51:00     +0139      0000   016093     22000
2016-01-01 03:51:00     +0133      0000   016093     22000


In [16]:
sand_df.dropna(inplace=True)
replace_nines(sand_df)
sand_df.isna().sum()

sand_temp       0
sand_wind       9
sand_vis        4
sand_ceil    2379
dtype: int64

In [ ]:
# decided not to fill, because they are at "odd" times
# that don't align with other measurent times (:51 past each hour)

# sand_df.sand_ceil.fillna(method='ffill', inplace=True)
# sand_df.isna().sum()

In [20]:
sand_df.dropna(inplace=True)

In [21]:
summarize(sand_df)


Shape: (24007, 4)

  Nulls:
sand_temp    0
sand_wind    0
sand_vis     0
sand_ceil    0
dtype: int64

df.head()
                    sand_temp sand_wind sand_vis sand_ceil
datetime                                                  
2016-01-01 00:51:00     +0156      0026   016093     22000
2016-01-01 01:51:00     +0144      0021   016093     22000
2016-01-01 02:51:00     +0139      0000   016093     22000
2016-01-01 03:51:00     +0133      0000   016093     22000
2016-01-01 04:51:00     +0122      0000   016093     22000


In [22]:
sand_df.to_csv('../data/san_diego_weather2.csv')

---

## Riverside DataFrame

In [23]:
summarize(rive_df)


Shape: (33905, 5)

  Nulls:
ceil      0
stn_id    0
temp      0
vis       0
wind      0
dtype: int64

df.head()
                            ceil       stn_id     temp           vis  \
datetime                                                               
2016-01-01 00:53:00  22000,5,9,N  72286903171  +0133,5  016093,5,N,5   
2016-01-01 01:53:00  22000,5,9,N  72286903171  +0128,5  016093,5,N,5   
2016-01-01 02:53:00  22000,5,9,N  72286903171  +0122,5  016093,5,N,5   
2016-01-01 03:53:00  22000,5,9,N  72286903171  +0117,5  016093,5,N,5   
2016-01-01 04:53:00  22000,5,9,N  72286903171  +0111,5  016093,5,N,5   

                               wind  
datetime                             
2016-01-01 00:53:00  030,5,N,0051,5  
2016-01-01 01:53:00  040,5,N,0046,5  
2016-01-01 02:53:00  040,5,N,0046,5  
2016-01-01 03:53:00  060,5,N,0041,5  
2016-01-01 04:53:00  040,5,N,0051,5  


In [24]:
rive_df.drop_duplicates().shape

(17294, 5)

In [25]:
rive_df = rive_df.drop_duplicates()
rive_df.shape

(17294, 5)

In [26]:
extract_statistic(rive_df, 'temp', 'rive_temp', 0)
extract_statistic(rive_df, 'wind', 'rive_wind', 3)
extract_statistic(rive_df, 'vis',  'rive_vis',  0)
extract_statistic(rive_df, 'ceil', 'rive_ceil', 0)

rive_df.drop(columns=['stn_id'], inplace=True)

In [27]:
summarize(rive_df)


Shape: (17294, 4)

  Nulls:
rive_temp    0
rive_wind    0
rive_vis     0
rive_ceil    0
dtype: int64

df.head()
                    rive_temp rive_wind rive_vis rive_ceil
datetime                                                  
2016-01-01 00:53:00     +0133      0051   016093     22000
2016-01-01 01:53:00     +0128      0046   016093     22000
2016-01-01 02:53:00     +0122      0046   016093     22000
2016-01-01 03:53:00     +0117      0041   016093     22000
2016-01-01 04:53:00     +0111      0051   016093     22000


In [28]:
rive_df.dropna(inplace=True)
replace_nines(rive_df)
rive_df.isna().sum()

rive_temp      0
rive_wind    111
rive_vis      33
rive_ceil     51
dtype: int64

In [ ]:
# did not fill forward... small num of nulls

# rive_df.sand_COLUMN.fillna(method='ffill', inplace=True)
# rive_df.isna().sum()

In [29]:
rive_df.dropna(inplace=True)

In [30]:
summarize(rive_df)


Shape: (17142, 4)

  Nulls:
rive_temp    0
rive_wind    0
rive_vis     0
rive_ceil    0
dtype: int64

df.head()
                    rive_temp rive_wind rive_vis rive_ceil
datetime                                                  
2016-01-01 00:53:00     +0133      0051   016093     22000
2016-01-01 01:53:00     +0128      0046   016093     22000
2016-01-01 02:53:00     +0122      0046   016093     22000
2016-01-01 03:53:00     +0117      0041   016093     22000
2016-01-01 04:53:00     +0111      0051   016093     22000


In [31]:
rive_df.to_csv('../data/riverside_weather2.csv')

## Redding DataFrame

In [32]:
summarize(redd_df)


Shape: (41245, 5)

  Nulls:
ceil      0
stn_id    0
temp      0
vis       0
wind      0
dtype: int64

df.head()
                            ceil       stn_id     temp           vis  \
datetime                                                               
2016-01-01 00:00:00  99999,9,9,N  72592024257  +0111,1  016000,1,9,9   
2016-01-01 00:53:00  22000,5,9,N  72592024257  +0094,5  016093,5,N,5   
2016-01-01 01:53:00  22000,5,9,N  72592024257  +0083,5  016093,5,N,5   
2016-01-01 02:53:00  22000,5,9,N  72592024257  +0078,5  016093,5,N,5   
2016-01-01 03:53:00  22000,5,9,N  72592024257  +0067,5  016093,5,N,5   

                               wind  
datetime                             
2016-01-01 00:00:00  010,1,N,0088,1  
2016-01-01 00:53:00  360,5,N,0072,5  
2016-01-01 01:53:00  010,5,N,0093,5  
2016-01-01 02:53:00  360,5,N,0067,5  
2016-01-01 03:53:00  360,5,N,0072,5  


In [33]:
redd_df.drop_duplicates().shape

(25646, 5)

In [34]:
redd_df = redd_df.drop_duplicates()
redd_df.shape

(25646, 5)

In [35]:
extract_statistic(redd_df, 'temp', 'redd_temp', 0)
extract_statistic(redd_df, 'wind', 'redd_wind', 3)
extract_statistic(redd_df, 'vis',  'redd_vis',  0)
extract_statistic(redd_df, 'ceil', 'redd_ceil', 0)

redd_df.drop(columns=['stn_id'], inplace=True)

In [36]:
summarize(redd_df)


Shape: (25582, 4)

  Nulls:
redd_temp    32
redd_wind     0
redd_vis      0
redd_ceil     0
dtype: int64

df.head()
                    redd_temp redd_wind redd_vis redd_ceil
datetime                                                  
2016-01-01 00:00:00     +0111      0088   016000     99999
2016-01-01 00:53:00     +0094      0072   016093     22000
2016-01-01 01:53:00     +0083      0093   016093     22000
2016-01-01 02:53:00     +0078      0067   016093     22000
2016-01-01 03:53:00     +0067      0072   016093     22000


In [37]:
redd_df.dropna(inplace=True)
replace_nines(redd_df)
redd_df.isna().sum()

redd_temp       0
redd_wind     194
redd_vis       18
redd_ceil    3170
dtype: int64

In [39]:
# decided not to fill, because they are at "odd" times
# that don't align with other measurent times (:53 past each hour)

# redd_df.sand_ceil.fillna(method='ffill', inplace=True)
# redd_df.isna().sum()

In [40]:
redd_df.dropna(inplace=True)

In [41]:
summarize(redd_df)


Shape: (22201, 4)

  Nulls:
redd_temp    0
redd_wind    0
redd_vis     0
redd_ceil    0
dtype: int64

df.head()
                    redd_temp redd_wind redd_vis redd_ceil
datetime                                                  
2016-01-01 00:53:00     +0094      0072   016093     22000
2016-01-01 01:53:00     +0083      0093   016093     22000
2016-01-01 02:53:00     +0078      0067   016093     22000
2016-01-01 03:53:00     +0067      0072   016093     22000
2016-01-01 05:53:00     +0056      0072   016093     22000


In [42]:
redd_df.to_csv('../data/redding_weather2.csv')

## Fresno DataFrame

In [43]:
summarize(fres_df)


Shape: (37439, 5)

  Nulls:
ceil      0
stn_id    0
temp      0
vis       0
wind      0
dtype: int64

df.head()
                            ceil       stn_id     temp           vis  \
datetime                                                               
2016-01-01 00:00:00  99999,9,9,N  72389093193  +0106,1  011000,1,9,9   
2016-01-01 00:53:00  22000,5,9,N  72389093193  +0100,5  011265,5,N,5   
2016-01-01 01:53:00  22000,5,9,N  72389093193  +0078,5  011265,5,N,5   
2016-01-01 02:53:00  22000,5,9,N  72389093193  +0067,5  009656,5,N,5   
2016-01-01 03:53:00  22000,5,9,N  72389093193  +0056,5  008047,5,N,5   

                               wind  
datetime                             
2016-01-01 00:00:00  290,1,N,0015,1  
2016-01-01 00:53:00  999,9,C,0000,5  
2016-01-01 01:53:00  999,9,C,0000,5  
2016-01-01 02:53:00  999,9,C,0000,5  
2016-01-01 03:53:00  310,5,N,0015,5  


In [44]:
fres_df.drop_duplicates().shape

(24710, 5)

In [45]:
fres_df = fres_df.drop_duplicates()
fres_df.shape

(24710, 5)

In [46]:
extract_statistic(fres_df, 'temp', 'fres_temp', 0)
extract_statistic(fres_df, 'wind', 'fres_wind', 3)
extract_statistic(fres_df, 'vis',  'fres_vis',  0)
extract_statistic(fres_df, 'ceil', 'fres_ceil', 0)

fres_df.drop(columns=['stn_id'], inplace=True)

In [47]:
summarize(fres_df)


Shape: (24676, 4)

  Nulls:
fres_temp    17
fres_wind     0
fres_vis      0
fres_ceil     0
dtype: int64

df.head()
                    fres_temp fres_wind fres_vis fres_ceil
datetime                                                  
2016-01-01 00:00:00     +0106      0015   011000     99999
2016-01-01 00:53:00     +0100      0000   011265     22000
2016-01-01 01:53:00     +0078      0000   011265     22000
2016-01-01 02:53:00     +0067      0000   009656     22000
2016-01-01 03:53:00     +0056      0015   008047     22000


In [48]:
fres_df.dropna(inplace=True)
replace_nines(fres_df)
fres_df.isna().sum()

fres_temp       0
fres_wind      22
fres_vis        5
fres_ceil    2927
dtype: int64

In [51]:
# decided not to fill, because they are at "odd" times
# that don't align with other measurent times (:53 past each hour)

# fres_df.sand_COLUMN.fillna(method='ffill', inplace=True)
# fres_df.isna().sum()

In [52]:
fres_df.dropna(inplace=True)

In [53]:
summarize(fres_df)


Shape: (21711, 4)

  Nulls:
fres_temp    0
fres_wind    0
fres_vis     0
fres_ceil    0
dtype: int64

df.head()
                    fres_temp fres_wind fres_vis fres_ceil
datetime                                                  
2016-01-01 00:53:00     +0100      0000   011265     22000
2016-01-01 01:53:00     +0078      0000   011265     22000
2016-01-01 02:53:00     +0067      0000   009656     22000
2016-01-01 03:53:00     +0056      0015   008047     22000
2016-01-01 04:53:00     +0039      0000   006437     22000


In [54]:
fres_df.to_csv('../data/fresno_weather2.csv')